In [1]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
movie_data = load_files(r"D:\Master Thesis\txt_sentoken")
movie_reviews, y = movie_data.data, movie_data.target

In [3]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(movie_reviews)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(movie_reviews[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()

    document = document.split()

    # normalize whitespace
    document = ' '.join(document)
    
    #documents.append(document)
    documents.append(str(movie_reviews[sen]))

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

doc = documents[413]

from lime import lime_text
from sklearn.pipeline import make_pipeline

c = make_pipeline(vectorizer, classifier)

In [8]:
y_pred = classifier.predict(X_test)

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[136  72]
 [ 73 119]]
              precision    recall  f1-score   support

           0       0.65      0.65      0.65       208
           1       0.62      0.62      0.62       192

    accuracy                           0.64       400
   macro avg       0.64      0.64      0.64       400
weighted avg       0.64      0.64      0.64       400

0.6375


In [10]:
from sklearn.tree import export_text
r = export_text(classifier, feature_names=vectorizer.get_feature_names())
print(r)

|--- bad <= 0.04
|   |--- worst <= 0.05
|   |   |--- boring <= 0.04
|   |   |   |--- wasted <= 0.04
|   |   |   |   |--- ridiculous <= 0.03
|   |   |   |   |   |--- awful <= 0.04
|   |   |   |   |   |   |--- performances <= 0.02
|   |   |   |   |   |   |   |--- minutes <= 0.05
|   |   |   |   |   |   |   |   |--- lame <= 0.06
|   |   |   |   |   |   |   |   |   |--- filmmakers <= 0.06
|   |   |   |   |   |   |   |   |   |   |--- turns <= 0.08
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 26
|   |   |   |   |   |   |   |   |   |   |--- turns >  0.08
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |   |   |   |   |   |--- filmmakers >  0.06
|   |   |   |   |   |   |   |   |   |   |--- making <= 0.03
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |   |   |   |   |   |   |--- making >  0.03
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   